<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">How can we populate a Time-Variant Property with a value for a given entity e.g. Instrument using the LUSID APIs via the LUSID website?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">How can we populate a Time-Variant Property with a value for a given entity e.g. Instrument using the LUSID APIs via the Python SDK?</p></li>
</ul>

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Populating Time-Variant Property Values</p>

In [2]:
import pprint
import os
import lusid
from lusidjam import RefreshingToken
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusid.models import *

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = SyncApiClientFactory(config_loaders=config_loaders)

instruments_api = api_factory.build(lusid.api.InstrumentsApi)
display(instruments_api)

Unable to open secrets file c:\Users\MollyMcDonagh\Dev\notebooks\public\V2\university\secrets.json


In [3]:
instruments_api.upsert_instruments(
    scope="FinbourneUniversity",
    request_body={
        "instr_1": lusid.InstrumentDefinition(
            name="MICROSOFT CORP",
            identifiers={
                "Figi": lusid.InstrumentIdValue(
                    value="BBG000BPH459"
                )
            }
        )
    }
)

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://mollyfinbourne.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://mollyfinbourne.lusid.com/app/insights/logs/0HN9I4LJ4FCCF:00000015',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {'actions': [{'description': 'The request identifiers of '
                                          'Unchanged Instruments',
                           'identifierType': 'RequestId',
                           'identifiers': ['instr_1'],
                           'type': 'UnchangedInstruments'}]},
 'staged': {},
 'values': {'instr_1': {'assetClass': 'Unknown',
                        'domCcy': 'ZZZ',
                        'href': 'htt

In [4]:
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

try:
    property_definitions_api.delete_property_definition(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="MarketCapitalisation")
except:
    pass

issuer_prop_definition = property_definitions_api.create_property_definition(
    create_property_definition_request=lusid.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="MarketCapitalisation",
        display_name="Market Capitalisation",
        property_description="The market capitalisation of the instrument",
        data_type_id=lusid.ResourceId(
            scope="system",
            code="currencyAndAmount"
        ),
        life_time="TimeVariant",
        constraint_style="Property"
    )
)

In [5]:
result_1 = instruments_api.upsert_instruments_properties(
    scope="FinbourneUniversity",
    upsert_instrument_property_request=[
        lusid.UpsertInstrumentPropertyRequest(
            identifier_type="Figi",
            identifier="BBG000BPH459",
            properties=[
                lusid.ModelProperty(
                    key="Instrument/FinbourneUniversity/MarketCapitalisation",
                    value=lusid.PropertyValue(
                        metric_value=lusid.MetricValue(
                            value=780360000000,
                            unit="USD"
                        )
                    )
                )
            ]
        )
    ]
)

print(result_1)

{'as_at_date': datetime.datetime(2025, 1, 13, 13, 49, 37, 83396, tzinfo=datetime.timezone.utc),
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://mollyfinbourne.lusid.com/app/insights/logs/0HN9JQH7C8736:0000001F',
            'method': 'GET',
            'relation': 'RequestLogs'}]}


<img src="./data/TimeVariantProperties-1.png" width="1200"/>

<img src="./data/TimeVariantProperties-1-filled.png" width="800"/>

In [7]:
result_2 = instruments_api.upsert_instruments_properties(
    scope="FinbourneUniversity",
    upsert_instrument_property_request=[
        lusid.UpsertInstrumentPropertyRequest(
            identifier_type="Figi",
            identifier="BBG000BPH459",
            properties=[
                lusid.ModelProperty(
                    key="Instrument/FinbourneUniversity/MarketCapitalisation",
                    value=lusid.PropertyValue(
                        metric_value=lusid.MetricValue(
                            value=1200000000000,
                            unit="USD"
                        )
                    ),
                    effective_from="2020-01-01T00:00:00Z"
                )
            ]
        )
    ]
)

print(result_2)

{'as_at_date': datetime.datetime(2025, 1, 13, 13, 50, 52, 262588, tzinfo=datetime.timezone.utc),
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://mollyfinbourne.lusid.com/app/insights/logs/0HN9JOG6C34I0:00000015',
            'method': 'GET',
            'relation': 'RequestLogs'}]}


<img src="./data/TimeVariantProperties-2.png" width="1200"/>

<img src="./data/TimeVariantProperties-2-filled.png" width="800"/>

In [9]:
result_3 = instruments_api.upsert_instruments_properties(
    scope="FinbourneUniversity",
    upsert_instrument_property_request=[
        lusid.UpsertInstrumentPropertyRequest(
            identifier_type="Figi",
            identifier="BBG000BPH459",
            properties=[
                lusid.ModelProperty(
                    key="Instrument/FinbourneUniversity/MarketCapitalisation",
                    value=lusid.PropertyValue(
                        metric_value=lusid.MetricValue(
                            value=1680000000000,
                            unit="USD"
                        )
                    ),
                    effective_from="2021-01-01T00:00:00Z",
                    effective_until="2021-12-31T00:00:00Z"
                )
            ]
        )
    ]
)

print(result_3)

{'as_at_date': datetime.datetime(2025, 1, 13, 13, 51, 19, 954096, tzinfo=datetime.timezone.utc),
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://mollyfinbourne.lusid.com/app/insights/logs/0HN9JQ0FNINJR:0000019E',
            'method': 'GET',
            'relation': 'RequestLogs'}]}


<img src="./data/TimeVariantProperties-3.png" width="1200"/>

<img src="./data/TimeVariantProperties-3-filled.png" width="800"/>

In [10]:
instruments_api.upsert_instruments_properties(
    scope="FinbourneUniversity",
    upsert_instrument_property_request=[
        lusid.UpsertInstrumentPropertyRequest(
            identifier_type="Figi",
            identifier="BBG000BPH459",
            properties=[
                lusid.ModelProperty(
                    key="Instrument/FinbourneUniversity/MarketCapitalisation",
                    value=None,
                    effective_until="2018-12-31T00:00:00Z"
                )
            ]
        )
    ]
)

result_4 = instruments_api.upsert_instruments_properties(
    scope="FinbourneUniversity",
    upsert_instrument_property_request=[
        lusid.UpsertInstrumentPropertyRequest(
            identifier_type="Figi",
            identifier="BBG000BPH459",
            properties=[
                lusid.ModelProperty(
                    key="Instrument/FinbourneUniversity/MarketCapitalisation",
                    value=None,
                    effective_from="2022-01-01T00:00:00Z"
                )
            ]
        )
    ]
)

print(result_4)

{'as_at_date': datetime.datetime(2025, 1, 13, 13, 51, 38, 473572, tzinfo=datetime.timezone.utc),
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://mollyfinbourne.lusid.com/app/insights/logs/0HN9JM9BLN1V3:0000016F',
            'method': 'GET',
            'relation': 'RequestLogs'}]}


<img src="./data/TimeVariantProperties-4.png" width="1200"/>

<img src="./data/TimeVariantProperties-4-filled.png" width="800"/>

In [11]:
property_definitions_api.delete_property_definition(
    domain="Instrument",
    scope="FinbourneUniversity",
    code="MarketCapitalisation")

{'asAt': datetime.datetime(2025, 1, 13, 13, 51, 41, 879771, tzinfo=datetime.timezone.utc),
 'effectiveFrom': datetime.datetime(1, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'entityType': 'PropertyDefinition',
 'entityUniqueId': None,
 'href': 'https://mollyfinbourne.lusid.com/api/api/entities/propertydefinitions/?effectiveAt=0001-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2025-01-13T13%3A51%3A41.8797710%2B00%3A00',
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://mollyfinbourne.lusid.com/app/insights/logs/0HN9JQ0FNIO31:0000005D',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'stagedModifications': {'countPending': 0,
                         'hrefPending': 'https://mollyfinbourne.lusid.com/api/api/stagedmodifications/?asAt=2025-01-13T13%3A51%3A41.8797710%2B00%3A00&filter=entityUniqueId%20eq%20%2770714ac7-7f40-4078-aba5-b92c2f7f29dc%27%20and%20st

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What have we covered?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We populated the Microsoft Equity Instrument with a value for the Time-Variant "MarketCapitalisation" Property using the LUSID APIs via the LUSID website.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We populated the Microsoft Equity Instrument with a value for the Time-Variant "MarketCapitalisation" Property using the LUSID APIs via the Python SDK.</p></li>
</ul>